In [ ]:
import pycrfsuite

In [3]:
dataset_path = "C:\\Users\\mvy48\\Downloads\\ml_combined_anoop-cc-gokul_07Dec19.txt"
with open(dataset_path, "r", encoding="utf-8") as file:
    dataset = file.readlines()

In [4]:
dataset = dataset[:200000]

In [5]:
# Preprocess the dataset
sentences = []
for i, text in enumerate(dataset):
    print("Document " + str(i + 1) + " of " + str(len(dataset)), end="\r", flush=True)
    # Convert text to lowercase and remove leading/trailing whitespaces
    prepared_text = text.strip().lower()
    # Remove non-alphabetic characters except spaces
    prepared_text = "".join([c if c.isalpha() or c.isspace() else "" for c in prepared_text])
    sentences.append(prepared_text)

In [6]:
# Remove very short sentences
sentences = [s for s in sentences if len(s) > 5]


In [7]:
# Prepare training data
prepared_sentences = []
for sentence in sentences:
    # Calculate the positions of word boundaries
    lengths = [len(w) for w in sentence.split(" ")]
    positions = []
    next_pos = 0
    for length in lengths:
        next_pos = next_pos + length
        positions.append(next_pos)
    # Concatenate the sentence and remove spaces
    concatenated = sentence.replace(" ", "")
    # Create lists of characters and labels
    chars = [c for c in concatenated]
    labels = [0 if not i in positions else 1 for i, c in enumerate(concatenated)]
    prepared_sentences.append(list(zip(chars, labels)))
print([d for d in prepared_sentences[100]])


[('അ', 0), ('വ', 0), ('ഹ', 0), ('ത', 0), ('ബ', 1), ('ന', 0), ('ധ', 0), ('ത', 0), ('ത', 0), ('ല', 0), ('അ', 1), ('സ', 0), ('വ', 0), ('ര', 0), ('സ', 0), ('യ', 0), ('ങ', 0), ('ങ', 0), ('ള', 0), ('ണ', 0), ('ശ', 1), ('ല', 0), ('ജ', 0), ('ത', 1), ('ര', 0), ('വ', 0), ('ദ', 0), ('യ', 0), ('ട', 0), ('ദ', 1), ('ര', 0), ('ണ', 0), ('മ', 0), ('യ', 0), ('ക', 1), ('ല', 0), ('പ', 0), ('ത', 0), ('ക', 0), ('ത', 0), ('ത', 0), ('ന', 0), ('ക', 1), ('ര', 0), ('ണ', 0), ('മ', 0), ('യ', 0), ('ത', 0)]


In [8]:
# Define functions to create features and labels
def create_char_features(sentence, i):
    features = [
        'bias',
        'char=' + sentence[i][0] 
    ]
    if i >= 1:
        features.extend([
            'char-1=' + sentence[i-1][0],
            'char-1:0=' + sentence[i-1][0] + sentence[i][0],
        ])
    else:
        features.append("BOS")
    if i >= 2:
        features.extend([
            'char-2=' + sentence[i-2][0],
            'char-2:0=' + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-2:-1=' + sentence[i-2][0] + sentence[i-1][0],
        ])
    if i >= 3:
        features.extend([
            'char-3:0=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-3:-1=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0],
        ])
    if i + 1 < len(sentence):
        features.extend([
            'char+1=' + sentence[i+1][0],
            'char:+1=' + sentence[i][0] + sentence[i+1][0],
        ])
    else:
        features.append("EOS")
    if i + 2 < len(sentence):
        features.extend([
            'char+2=' + sentence[i+2][0],
            'char:+2=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0],
            'char+1:+2=' + sentence[i+1][0] + sentence[i+2][0],
        ])
    if i + 3 < len(sentence):
        features.extend([
            'char:+3=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0]+ sentence[i+3][0],
            'char+1:+3=' + sentence[i+1][0] + sentence[i+2][0] + sentence[i+3][0],
        ])
    return features

In [9]:
def create_sentence_features(prepared_sentence):
    return [create_char_features(prepared_sentence, i) for i in range(len(prepared_sentence))]


In [10]:
def create_sentence_labels(prepared_sentence):
    return [str(part[1]) for part in prepared_sentence]

In [11]:
# Create features and labels for training data
X = [create_sentence_features(ps) for ps in prepared_sentences]
y = [create_sentence_labels(ps)   for ps in prepared_sentences]

In [12]:
split_index = int(0.7 * len(X))
X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

In [13]:
# Train a CRF model
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
trainer.set_params({
    'c1': 1.0, 
    'c2': 1e-3,
    'max_iterations': 60,
    'feature.possible_transitions': True
})
trainer.train('trained_model.crfsuite')

In [14]:
# Evaluate the trained model
tagger = pycrfsuite.Tagger()
tagger.open('trained_model.crfsuite')

In [15]:
def segment_sentence(sentence):
    sent = sentence.replace(" ", "")
    prediction = tagger.tag(create_sentence_features(sent))
    complete = ""
    for i, p in enumerate(prediction):
        if p == "1":
            complete += " " + sent[i]
        else:
            complete += sent[i]
    return complete
print(segment_sentence("മകനുവേണ്ടിഅച്ഛനുംപ്രമുഖനിർമ്മാതാവുമായഎഎൽഅളഗപ്പനാണ്വിവാഹംനടത്താൻതീരുമാനിച്ചിരിക്കുന്നത്എന്നാണ്സൂചന."))
print(segment_sentence("ഇന്ന്റായ്ബറേലിയിൽകോൺഗ്രസ്ഉപാദ്ധ്യക്ഷൻരാഹുൽഗാന്ധിയുടെരണ്ട്റാലികളിൽപ്രിയങ്കഗാന്ധിയുംപങ്കെടുക്കും."))
print(segment_sentence("എന്നാൽരണ്ടുമാസത്തിനകംഭരണഘടനാകോടതിഅദ്ദേഹത്തെപ്രസിഡൻറ്പദത്തിൽവീണ്ടുംഅവരോധിച്ചു."))
print(segment_sentence("വിവോഇന്ത്യപ്രീമിയർലീഗിൽഔദ്യോഗികപാർട്ണറായിരുന്നുടാറ്റാനെക്സോൺ."))
print(segment_sentence("വലിയൊരുസ്വപ്നംപൊലിഞ്ഞതിൽസങ്കടമൊക്കെയുണ്ടായെങ്കിലുംഅത്മുഹമ്മദ്താലിഫ്എന്നപന്ത്രണ്ടുകാരനെപ്രസിദ്ധനാക്കിയിരിക്കുകയാണ്."))
print(segment_sentence("ഏറെനാളത്തെപ്രണയത്തിന്ശേഷമാണ്മലയാളികളുടെപ്രിയനടിഭാവനയുംകന്നഡസിനിമാനിർമാതാവ്നവീനുംവിവാഹിതരാകാൻപോകുന്നത്."))
print(segment_sentence("മൂന്നാംവിക്കറ്റിൽരോഹിത്-റായിഡുസഖ്യംകൂട്ടിച്ചേർച്ച#റൺസാണ്ഇന്ത്യൻഇന്നിംഗ്സിന്റെനട്ടെല്ല്."))
print(segment_sentence("എന്നെപ്പോലെതന്നെഅവരുംവിശന്നുകൊണ്ടായിരിക്കുംഉറങ്ങാൻപോകുന്നത്എന്നെല്ലാംഞാൻചിന്തിച്ചിരുന്നു."))
print(segment_sentence("ആറ്മുതൽഒൻപത്സീറ്റുകൾവരെയുള്ളഒരുചെറിയവിമാനംചാർട്ടർചെയ്ത്പറക്കാൻമണിക്കൂറിന്ഒന്നരലക്ഷംമുതൽരണ്ട്ലക്ഷംവരെയാണ്ഇപ്പോൾചെലവ്വരുന്നത്."))
print(segment_sentence("ഇതേതുടർന്ന്ഡിജിപിയുംമന്ത്രികടകംപള്ളിസുരേന്ദ്രനുംമാധ്യമങ്ങൾക്ക്വിലക്കില്ലെന്നുംസുരക്ഷയൊരുക്കുന്നതിൻറെഭാഗമായനടപടിമാത്രമാണ്നടക്കുന്നതെന്നുംവിശദീകരിച്ചിരുന്നു."))
print(segment_sentence("വിശദീകരിക്കുന്നഎല്ലാംതന്റെനിരവധിസന്ദർഭങ്ങളിൽപ്രയോഗിക്കുന്നതിനായിഞാൻസന്തുലിതമാണ്"))

മകന ുവേണ്ടി അച്ഛന ുംപ്രമുഖന ിർമ്മാതാവുമായ എഎൽ അളഗ പ്പനാണ്വിവാഹംനടത്താൻതീരുമാനിച്ചിരിക്കുന്നത് എന്നാണ്സൂചന.
ഇന്ന്റായ്ബറേലിയി ൽകോൺഗ്രസ് ഉപാദ്ധ്യക്ഷൻ രാഹു ൽഗാന്ധിയുടെരണ്ട്റാലികളി ൽപ്രിയങ്കഗാന്ധിയുംപങ്കെടുക്കും.
എന്നാ ൽരണ്ടുമാസത്തിനകം ഭരണഘടനാകോടതി അദ്ദേഹത്തെപ്രസിഡൻറ ്പദത്തി ൽവീണ്ടും അവര ോധിച്ചു.
വിവോ ഇന്ത്യപ്രീമിയർലീഗി ൽ ഔദ്യോഗികപാർട്ണറായിരുന്നുടാറ്റാനെക്സോൺ.
വലിയൊരുസ്വപ്നംപൊലി ഞ്ഞതി ൽസങ് കടമൊക്കെയുണ്ടായെങ്കിലും അത്മുഹമ് മദ്താലിഫ് എന്നപന്ത്രണ്ടുകാരനെപ്രസിദ്ധനാക്കിയിരിക്കുകയാണ്.
ഏറെനാളത്തെപ്രണയത്തിന്ശേഷമാണ് മലയാളികളുടെപ്രിയനടിഭാവനയുംകന്ന ഡസിനിമാനിർമാതാവ്നവീനുംവിവാഹിതരാകാൻ പോകുന്നത്.
മൂന്നാംവിക്കറ്റി ൽരോഹിത്-റായിഡുസഖ്യംകൂട്ടിച്ചേർച്ച# റൺസ ാണ് ഇന്ത്യൻ ഇന്നിംഗ്സിന്റെനട്ടെല്ല്.
എന്നെപ്പോലെതന്നെ അവര ുംവിശന്നുകൊണ്ടായിരിക്കും ഉറങ ്ങാൻ പോകുന്നത് എന്നെല്ലാംഞാൻ ചിന്തിച്ചിരുന്നു.
ആറ്മുതൽ ഒൻപത്സീറ്റുകൾ വരെയുള്ള ഒര ുചെറിയവിമാനം ചാർട ്ടർ ചെയ്ത് പറക്കാൻമണിക്കൂറിന് ഒന്നരലക്ഷംമുതൽ രണ്ട്ലക്ഷംവരെയാണ് ഇപ്പോൾ ചെലവ്വരുന്നത്.
ഇതേതുടർന്ന്ഡിജിപിയുംമന്ത്രി കടകംപള്ളിസുരേന്ദ്രനുംമാധ്യമങ്ങൾ ക്ക്വിലക്കില്ലെന്നുംസുരക്ഷയൊ

In [16]:
tp = 0
fp = 0
fn = 0
n_correct = 0
n_incorrect = 0
for i in range(len(X_test)):
    prediction = tagger.tag(X_test[i])
    correct = y_test[i]
    zipped = list(zip(prediction, correct))
    tp += len([_ for l, c in zipped if l == c and l == "1"])
    fp += len([_ for l, c in zipped if l == "1" and c == "0"])
    fn += len([_ for l, c in zipped if l == "0" and c == "1"])
    n_incorrect += len([_ for l, c in zipped if l != c])
    n_correct += len([_ for l, c in zipped if l == c])

In [17]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = n_correct / (n_correct + n_incorrect)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:\t" + str(precision))
print("Recall:\t\t" + str(recall))
print("Accuracy:\t" + str(accuracy))
print("F1 score:\t" + str(f1_score))


Precision:	0.9396259398556203
Recall:		0.9302941822069429
Accuracy:	0.976900621369989
F1 score:	0.9349367761594344
